In [1]:
import pandas as pd
from gensim.models.word2vec import Word2Vec

In [7]:
train_df = pd.read_pickle("./train_reduce.pkl")
# test_df = pd.read_pickle("../input/hw2pkl/test.pkl")
train_df.head()

,ari_time,ari_title,price
0,2006-10-27,balda say investor asks,0
1,2006-11-01,ace lift pay,0
2,2006-11-08,namibian investors buy absa 's,1
3,2006-11-09,firstrand say law may pare fees,0
4,2006-11-09,china stocks rise 30 % in #,0


In [10]:
corpus = pd.concat([train_df.ari_time, train_df.ari_title])
print(corpus)
# corpus = pd.concat([train_df.ari_time, train_df.ari_title]).sample(frac=1)
corpus.head()

0                                               2006-10-27
1                                               2006-11-01
2                                               2006-11-08
3                                               2006-11-09
4                                               2006-11-09
5                                               2006-11-16
6                                               2006-11-22
7                                               2006-11-22
8                                               2006-11-29
9                                               2006-12-11
10                                              2006-12-11
11                                              2006-12-12
12                                              2006-12-14
13                                              2006-12-18
14                                              2006-12-19
15                                              2007-01-10
16                                              2007-01-

0    2006-10-27
1    2006-11-01
2    2006-11-08
3    2006-11-09
4    2006-11-09
dtype: object

In [4]:
%%time
# CPU times: user 1min 11s
model = Word2Vec(corpus, size=50, iter=10)
model.save('word2vec.model')

CPU times: user 4.9 s, sys: 68.7 ms, total: 4.97 s
Wall time: 2.46 s
